<a href="https://colab.research.google.com/github/athiagarajan/llm_gpt4all/blob/main/gpt4allindia1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install poppler-utils


In [ ]:

!pip install -Uqqq pip --progress-bar off
!pip install -qqq langchain==0.0.173 --progress-bar off
!pip install -qqq chromadb==0.3.23 --progress-bar off
!pip install -qqq pypdf==3.8.1 --progress-bar off
!pip install -qqq pygpt4all==1.1.0 --progress-bar off
!pip install -qqq pdf2image==1.16.3 --progress-bar off

In [ ]:

!gdown 1DpFisoGXsQbpQJvijuvxkLW_pg-FUUMF

In [ ]:
!wget https://gpt4all.io/models/ggml-gpt4all-j-v1.3-groovy.bin

In [4]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import GPT4All
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from pdf2image import convert_from_path

In [5]:

images = convert_from_path("ms-financial-statement.pdf", dpi=88)
len(images)

2

In [ ]:

images[0]

In [ ]:
images[1]

In [ ]:
loader = PyPDFLoader("ms-financial-statement.pdf")
documents = loader.load_and_split()
len(documents)

In [ ]:
print(documents[0].page_content)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
texts = text_splitter.split_documents(documents)
len(texts)
print(texts[0].page_content)


In [8]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = Chroma.from_documents(texts, embeddings, persist_directory="db")

In [9]:
model_n_ctx = 1000
model_path = "./ggml-gpt4all-j-v1.3-groovy.bin"
llm = GPT4All(model=model_path, n_ctx=1000, backend="gptj", verbose=False)

In [10]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True,
    verbose=False,
)


In [ ]:
%%time
res = qa(
    "How much is the dividend per share during during 2022? Extract it from the text."
)
res
print(res["result"])


In [ ]:
%%time
prompt = f"""How much is the investment amount in Microsoft on 6/22? Extract the answer from the text."""
res = qa(prompt.strip())
res
print(res["result"])